# Capacity Factors Forecast with Regression

In [11]:
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
from enum import Enum

from ngboost.scores import LogScore
from shapely.geometry import Point
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="darkgrid")

from ngboost import NGBRegressor
from sklearn.model_selection import train_test_split
from ngboost.distns import Exponential, Normal, LogNormal

from sklearn.metrics import mean_pinball_loss
from sklearn.metrics import mean_squared_error

from scipy.stats import norm
import ephem
from datetime import datetime
import pickle
from pathlib import Path

paths = {"era5_eu_2013": "resources/europe-2013-era5.nc",
         "era5_tutorial": "resources/europe-2013-era5-tutorial.nc",
         "offshore_shape": "resources/regions_offshore_elec_s_37.geojson",
         "onshore_shape": "resources/regions_onshore_elec_s_37.geojson",
         "capfacs": "resources/capfacs_37.csv",
         "era5_regions": "resources/europe-2013-era5-regions.nc"}

In [12]:
ds = xr.open_dataset(filename_or_obj=paths["era5_regions"], engine="netcdf4")
ds

<xarray.Dataset>
Dimensions:           (region: 65, time: 8760)
Coordinates:
  * region            (region) object 'AL0 0 on' 'AT0 0 on' ... 'SI0 0 off'
  * time              (time) datetime64[ns] 2013-01-01 ... 2013-12-31T23:00:00
Data variables:
    height            (region, time) float64 ...
    wnd100m           (region, time) float64 ...
    roughness         (region, time) float64 ...
    influx_toa        (region, time) float64 ...
    influx_direct     (region, time) float64 ...
    influx_diffuse    (region, time) float64 ...
    albedo            (region, time) float64 ...
    temperature       (region, time) float64 ...
    soil_temperature  (region, time) float64 ...
    runoff            (region, time) float64 ...
Attributes:
    description:  Era5 data with mean value of the coordinates within a region

In [13]:
capfacts = pd.read_csv(paths["capfacs"])
capfacts

,snapshot,AL0 0 offwind-ac,AL0 0 onwind,AL0 0 solar,AT0 0 onwind,AT0 0 ror,AT0 0 solar,BA0 0 onwind,BA0 0 solar,BE0 0 offwind-ac,...,SE4 0 onwind,SE4 0 ror,SE4 0 solar,SI0 0 offwind-ac,SI0 0 onwind,SI0 0 ror,SI0 0 solar,SK0 0 onwind,SK0 0 ror,SK0 0 solar
0,2013-01-01 00:00:00,0.003291,0.001469,0.0,0.163262,0.224456,0.0,0.007340,0.0,1.000000,...,0.459609,0.626955,0.0,0.000000,0.055146,0.344668,0.0,0.361009,0.106197,0.0
1,2013-01-01 01:00:00,0.002103,0.000000,0.0,0.171340,0.224369,0.0,0.007939,0.0,0.999998,...,0.463265,0.625502,0.0,0.000000,0.052605,0.344657,0.0,0.368912,0.106012,0.0
2,2013-01-01 02:00:00,0.000000,0.000000,0.0,0.171035,0.224300,0.0,0.007829,0.0,0.993941,...,0.463777,0.624810,0.0,0.000000,0.052222,0.344593,0.0,0.382949,0.105968,0.0
3,2013-01-01 03:00:00,0.000000,0.000000,0.0,0.169685,0.224249,0.0,0.005766,0.0,0.916094,...,0.463041,0.623794,0.0,0.000000,0.050762,0.344626,0.0,0.388344,0.106215,0.0
4,2013-01-01 04:00:00,0.000000,0.000000,0.0,0.159757,0.224213,0.0,0.004262,0.0,0.704786,...,0.457253,0.623085,0.0,0.000000,0.047285,0.344607,0.0,0.409303,0.106364,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2013-12-31 19:00:00,0.029235,0.024321,0.0,0.109127,0.275353,0.0,0.001076,0.0,0.998922,...,0.358191,0.469586,0.0,0.027877,0.001708,0.696975,0.0,0.125735,0.157729,0.0
8756,2013-12-31 20:00:00,0.034024,0.031357,0.0,0.120485,0.275159,0.0,0.001090,0.0,0.965757,...,0.324709,0.469395,0.0,0.023567,0.000000,0.696262,0.0,0.142431,0.157817,0.0
8757,2013-12-31 21:00:00,0.037104,0.034522,0.0,0.128186,0.274975,0.0,0.001062,0.0,0.852484,...,0.288415,0.469208,0.0,0.000000,0.004891,0.695524,0.0,0.153930,0.157899,0.0
8758,2013-12-31 22:00:00,0.029730,0.027419,0.0,0.124740,0.274784,0.0,0.001351,0.0,0.776442,...,0.268695,0.469028,0.0,0.000000,0.009091,0.694665,0.0,0.163442,0.157945,0.0


In [14]:
class EnergyType(Enum):
    """
    Represents the different type of renewable energy sources in pypsa-eur
    """
    OFFWIND_AC = "offwind-ac"
    OFFWIND_DC = "offwind-dc"
    ONWIND = "onwind-dc"
    SOLAR = "solar"
    ROR = "ror"
    NOT_DEFINED = "not_defined"

class Feature(Enum):
    """
    Represents the features, that can be extracted from the era-5 weather data set
    """
    HEIGHT = "height"
    WND100M = "wnd100m"
    ROUGHNESS = "roughness"
    INFLUX_TOA = "influx_toa"
    INFLUX_DIRECT = "influx_direct"
    INFLUX_DIFFUSE = "influx_diffuse"
    ALBEDO = "albedo"
    TEMPERATURE = "temperature"
    SOIL_TEMPERATURE = "soil_temperature"
    RUNOFF = "runoff"

"""
Determines which features are selected to calculate the capacity factor of a certain energy type.
"""
feature_set = {
    EnergyType.OFFWIND_AC: [Feature.HEIGHT, Feature.WND100M, Feature.ROUGHNESS],
    EnergyType.OFFWIND_DC: [Feature.HEIGHT, Feature.WND100M, Feature.ROUGHNESS],
    EnergyType.ONWIND: [Feature.HEIGHT, Feature.WND100M, Feature.ROUGHNESS],
    EnergyType.SOLAR: [Feature.INFLUX_TOA, Feature.INFLUX_DIRECT, Feature.INFLUX_DIFFUSE, Feature.TEMPERATURE],
    EnergyType.ROR: []
}

def find_countries_in_capfacts(country_name="") -> list:
    """
    Returns the full region names and energy types of the given name abbreviation that can be found in the .csv file with capacity factors.
    :param country_name: Two character abbreviation of the searched country
    :return: list of all regions and energy types to the given country name
    """
    countries = []
    for column in capfacts:
        if column.find(country_name) >= 0:
            countries.append(column)
    return countries


def get_energy_type(name: str) -> EnergyType:
    """
    Returns the energy type for a given string
    :param name: energy type as string
    :return: energy type for the given string
    """
    match name:
        case "offwind-ac":
            return EnergyType.OFFWIND_AC
        case "offwind-dc":
            return EnergyType.OFFWIND_DC
        case "onwind":
            return EnergyType.ONWIND
        case "solar":
            return EnergyType.SOLAR
        case "ror":
            return EnergyType.ROR
        case _:
            return EnergyType.NOT_DEFINED

def get_ds_region_name(region_name: str, energy_type: EnergyType) -> str:
    """
    Returns the name or string that addresses the given region and energy type which can be used to address the data in the feature data set
    :param region_name: name of the region
    :param energy_type: the uses energy type in that region
    :return: string that can be used to fetch data from the feature data set
    """
    ds_region_name = region_name + " 0"
    if energy_type == EnergyType.ONWIND or energy_type == EnergyType.SOLAR or energy_type == EnergyType.ROR:
        ds_region_name += " on"
    elif energy_type == EnergyType.OFFWIND_AC or energy_type == EnergyType.OFFWIND_DC:
        ds_region_name += " off"
    else:
        ds_region_name += ""
    return ds_region_name

def parse_capfac_col(column_name: str) -> (str, EnergyType):
    """
    Returns a tuple of the region name and energy type for a given column name of the capfacts .csv file
    :param column_name: column name of the capfacts .csv file
    :return: Tuple of a region name and energy type, None if no region is found
    """
    col_args = column_name.split(" ")
    if len(col_args) == 3:
        region_name = col_args[0]
        energy_type = get_energy_type(col_args[2])
        return region_name, energy_type
    return None, None

def create_training_data_for_col(column_name: str) -> (np.ndarray, dict):
    """
    Creates and returns the the training data set with the relevant data for a given column name from the capfacts .csv file.
    The training data set is a tuple of a numpy array of capacity factors (target values) and a dictionary of the era5 data (feature data),
    :param column_name: column name of the capfacts .csv file
    :return: Tuple of capacity factor (Y) and trainings data (X)
            X                       : DataFrame object or List or numpy array of predictors (n x p) in Numeric format
            Y                       : DataFrame object or List or numpy array of outcomes (n) in Numeric format.
    """
    region_name, energy_type = parse_capfac_col(column_name)
    # print(region_name)
    # print(energy_type)
    ds_region_name = get_ds_region_name(region_name, energy_type)
    # print(ds_region_name)
    features = feature_set.get(energy_type)
    # print(features)

    Y_capfac = capfacts[column_name].values
    X = {}
    for feature in features:
        X[feature] = ds.sel(region=ds_region_name)[feature.value].values

    return Y_capfac, X


def shape_multi_feature_data(training_data: dict):
    """
    Reshapes the trainingsdata in an array of shape (n_samples, n_features)
    (8760, 2) ---> [[x_f1_1, x_f2_1], [x_f1_2, x_f2_2], ... , [x_f1_8760, x_f2_8760]]
    :param training_data as a dictinary of multiple 1-d arrays:
    :return: trainingsdata in array of shape (n_samples, n_features)
    """
    # tup = tuple(list(training_data.values()))
    # multi_feature_train_data = np.column_stack(tup)
    arrays = list(training_data.values())
    return np.stack(arrays, axis=-1)

In [15]:
def get_date_time_obj(date_time_str: str):
    # 2013-01-01 21:00:00
    return datetime.strptime(date_time_str, "%Y-%m-%d %H:%M:%S")

In [16]:
capfacts_columns = capfacts.columns.values


# capfacts_pred_q40 = pd.DataFrame(columns=capfacts_columns)
capfacts_pred_q40 = pd.DataFrame()
capfacts_pred_q40["snapshot"] = capfacts["snapshot"]

# capfacts_pred_q60 = pd.DataFrame(columns=capfacts_columns)
capfacts_pred_q60 = pd.DataFrame()
capfacts_pred_q60["snapshot"] = capfacts["snapshot"]

capfacts_pred_q40

,snapshot
0,2013-01-01 00:00:00
1,2013-01-01 01:00:00
2,2013-01-01 02:00:00
3,2013-01-01 03:00:00
4,2013-01-01 04:00:00
...,...
8755,2013-12-31 19:00:00
8756,2013-12-31 20:00:00
8757,2013-12-31 21:00:00
8758,2013-12-31 22:00:00


In [17]:
col_names = capfacts_columns[1:]

new_columns_q40 = {}
new_columns_q60 = {}

i = 1
for col in col_names:
    print("Processing \"", col, "\" (", i, "/", len(col_names), ")")
    i += 1
    region_name, energy_type = parse_capfac_col(col)

    if(energy_type == EnergyType.NOT_DEFINED):
        print("Skipped column: ", col)
        print("-------------------------------------------------------------------")
    elif(energy_type == EnergyType.ROR):
        print("Skipped column: ", col)
        print("-------------------------------------------------------------------")
    else:
        print("Create Trainings data for region: ", region_name, " with energy type: ", energy_type)

        Y, X = create_training_data_for_col(col)
        X_pred = shape_multi_feature_data(X)
        X_train, X_test, Y_train, Y_test = train_test_split(shape_multi_feature_data(X), Y, test_size=0.25, random_state=42)

        print("Fit Regression Model for region ", region_name)
        ngb = NGBRegressor(random_state=42, verbose=False).fit(X_train, Y_train)

        print("Predict capacity factors for region ", region_name)
        # Y_preds = ngb.predict(X_pred)
        Y_dists = ngb.pred_dist(X_pred)

        Y_preds_q40 = Y_dists.ppf(0.4)
        new_columns_q40[col] = pd.Series(Y_preds_q40)
        # capfacts_pred_q40[col] = Y_preds_q40

        Y_preds_q60 = Y_dists.ppf(0.6)
        new_columns_q60[col] = pd.Series(Y_preds_q60)
        # capfacts_pred_q60[col] = Y_preds_q60

        # print(capfacts_pred.head())
        print(col, " with q = 0.4")
        # print("Smallest value: ", capfacts_pred_q40[col].min())
        # print("Biggest value: ", capfacts_pred_q40[col].max())
        # print("#Negative Values: ", (capfacts_pred_q40[col] < 0).sum())
        # print("#Values > 1: ", (capfacts_pred_q40[col] > 1).sum())
        print("Smallest value: ", new_columns_q40[col].min())
        print("Biggest value: ", new_columns_q40[col].max())
        print("#Negative Values: ", (new_columns_q40[col] < 0).sum())
        print("#Values > 1: ", (new_columns_q40[col] > 1).sum())

        print("")
        print(col, " with q = 0.6")
        # print("Smallest value: ", capfacts_pred_q40[col].min())
        # print("Biggest value: ", capfacts_pred_q40[col].max())
        # print("#Negative Values: ", (capfacts_pred_q40[col] < 0).sum())
        # print("#Values > 1: ", (capfacts_pred_q40[col] > 1).sum())
        print("Smallest value: ", new_columns_q60[col].min())
        print("Biggest value: ", new_columns_q60[col].max())
        print("#Negative Values: ", (new_columns_q60[col] < 0).sum())
        print("#Values > 1: ", (new_columns_q60[col] > 1).sum())

        print("-------------------------------------------------------------------")

        print("")

new_columns_q40 = pd.DataFrame(new_columns_q40, index=capfacts_pred_q40.index)
capfacts_pred_q40 = pd.concat([capfacts_pred_q40, new_columns_q40], axis=1)
new_columns_q60 = pd.DataFrame(new_columns_q60, index=capfacts_pred_q60.index)
capfacts_pred_q60 = pd.concat([capfacts_pred_q60, new_columns_q60], axis=1)


Processing " AL0 0 offwind-ac " ( 1 / 150 )
Create Trainings data for region:  AL0  with energy type:  EnergyType.OFFWIND_AC
Fit Regression Model for region  AL0
Predict capacity factors for region  AL0
AL0 0 offwind-ac  with q = 0.4
Smallest value:  -0.012624098012820785
Biggest value:  0.9714872040522765
#Negative Values:  97
#Values > 1:  0

AL0 0 offwind-ac  with q = 0.6
Smallest value:  -0.006444241201678533
Biggest value:  1.0119108318100547
#Negative Values:  1
#Values > 1:  7
-------------------------------------------------------------------

Processing " AL0 0 onwind " ( 2 / 150 )
Create Trainings data for region:  AL0  with energy type:  EnergyType.ONWIND
Fit Regression Model for region  AL0
Predict capacity factors for region  AL0
AL0 0 onwind  with q = 0.4
Smallest value:  -0.006681087674393571
Biggest value:  0.6197746490529304
#Negative Values:  96
#Values > 1:  0

AL0 0 onwind  with q = 0.6
Smallest value:  -0.0028088120090494773
Biggest value:  0.6419837990087166
#Nega

In [18]:
capfacts_pred_q40.to_csv('results/capfacts_pred_q40.csv')
capfacts_pred_q60.to_csv('results/capfacts_pred_q60.csv')
capfacts_pred_q40

,snapshot,AL0 0 offwind-ac,AL0 0 onwind,AL0 0 solar,AT0 0 onwind,AT0 0 solar,BA0 0 onwind,BA0 0 solar,BE0 0 offwind-ac,BE0 0 offwind-dc,...,RS0 0 solar,SE4 0 offwind-ac,SE4 0 offwind-dc,SE4 0 onwind,SE4 0 solar,SI0 0 offwind-ac,SI0 0 onwind,SI0 0 solar,SK0 0 onwind,SK0 0 solar
0,2013-01-01 00:00:00,0.000928,0.018104,0.000975,0.175273,0.002074,0.007821,0.002178,0.985324,0.996833,...,0.001190,0.883585,0.929599,0.385410,0.001006,-0.000013,0.014425,0.001833,0.236722,0.000873
1,2013-01-01 01:00:00,0.000928,0.010446,0.000975,0.175273,0.002074,0.009557,0.001121,0.971108,0.996833,...,0.001190,0.864818,0.929391,0.385410,0.001006,-0.000013,0.019608,0.001833,0.236722,0.000873
2,2013-01-01 02:00:00,0.000928,0.004443,0.001260,0.152732,0.002074,0.009557,0.001121,0.937728,0.996629,...,0.001190,0.864818,0.929391,0.386180,0.001006,-0.000028,0.025489,0.001833,0.262663,0.000873
3,2013-01-01 03:00:00,0.000928,0.002503,0.002607,0.137212,0.002074,0.009557,0.001121,0.831023,0.938460,...,0.001190,0.864818,0.927961,0.386180,0.001006,-0.000023,0.036644,0.001833,0.262663,0.000873
4,2013-01-01 04:00:00,-0.000102,-0.000837,0.002607,0.130745,0.002074,0.010375,0.001121,0.645404,0.759356,...,0.001190,0.864818,0.929391,0.385410,0.001006,-0.000023,0.041910,0.001833,0.264453,0.000873
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2013-12-31 19:00:00,0.029153,0.052263,0.000961,0.111495,0.002074,0.001800,0.000976,0.919508,0.989035,...,0.000900,0.774638,0.858609,0.307886,0.001006,0.025999,0.003614,0.001377,0.074752,0.000940
8756,2013-12-31 20:00:00,0.029161,0.065263,0.000961,0.129865,0.002074,0.001800,0.000976,0.778407,0.910707,...,0.002864,0.737556,0.855220,0.283948,0.001006,0.021950,0.005001,0.001377,0.082766,0.000940
8757,2013-12-31 21:00:00,0.029153,0.070495,0.001175,0.129865,0.002074,0.001812,0.000976,0.707040,0.847579,...,0.010672,0.692466,0.791745,0.247635,0.001006,0.000050,0.005051,0.001377,0.083883,0.000940
8758,2013-12-31 22:00:00,0.029545,0.052263,0.001175,0.161829,0.002074,0.002011,0.000976,0.766593,0.919846,...,0.000017,0.672129,0.785177,0.204160,0.001006,-0.000171,0.008033,0.001377,0.090750,0.000940


In [19]:
cols_num = capfacts_pred_q40.select_dtypes(np.number).columns
print(cols_num)

capfacts_pred_q40[cols_num] = capfacts_pred_q40[cols_num].clip(lower=0, upper=1.02)
capfacts_pred_q60[cols_num] = capfacts_pred_q60[cols_num].clip(lower=0, upper=1.02)

# capfacts_pred_q40.iloc[:, 1:].clip(lower=0, upper=1.02)
# capfacts_pred_q60.iloc[:, 1:].clip(lower=0, upper=1.02)

capfacts_pred_q40.to_csv('results/capfacts_pred_q40_clipped.csv')
capfacts_pred_q60.to_csv('results/capfacts_pred_q60_clipped.csv')

# Alles klar, danke euch. Dann werde ich das auf [0, 1.02] beschränken
capfacts_pred_q40

Index(['AL0 0 offwind-ac', 'AL0 0 onwind', 'AL0 0 solar', 'AT0 0 onwind',
       'AT0 0 solar', 'BA0 0 onwind', 'BA0 0 solar', 'BE0 0 offwind-ac',
       'BE0 0 offwind-dc', 'BE0 0 onwind',
       ...
       'RS0 0 solar', 'SE4 0 offwind-ac', 'SE4 0 offwind-dc', 'SE4 0 onwind',
       'SE4 0 solar', 'SI0 0 offwind-ac', 'SI0 0 onwind', 'SI0 0 solar',
       'SK0 0 onwind', 'SK0 0 solar'],
      dtype='object', length=125)


,snapshot,AL0 0 offwind-ac,AL0 0 onwind,AL0 0 solar,AT0 0 onwind,AT0 0 solar,BA0 0 onwind,BA0 0 solar,BE0 0 offwind-ac,BE0 0 offwind-dc,...,RS0 0 solar,SE4 0 offwind-ac,SE4 0 offwind-dc,SE4 0 onwind,SE4 0 solar,SI0 0 offwind-ac,SI0 0 onwind,SI0 0 solar,SK0 0 onwind,SK0 0 solar
0,2013-01-01 00:00:00,0.000928,0.018104,0.000975,0.175273,0.002074,0.007821,0.002178,0.985324,0.996833,...,0.001190,0.883585,0.929599,0.385410,0.001006,0.000000,0.014425,0.001833,0.236722,0.000873
1,2013-01-01 01:00:00,0.000928,0.010446,0.000975,0.175273,0.002074,0.009557,0.001121,0.971108,0.996833,...,0.001190,0.864818,0.929391,0.385410,0.001006,0.000000,0.019608,0.001833,0.236722,0.000873
2,2013-01-01 02:00:00,0.000928,0.004443,0.001260,0.152732,0.002074,0.009557,0.001121,0.937728,0.996629,...,0.001190,0.864818,0.929391,0.386180,0.001006,0.000000,0.025489,0.001833,0.262663,0.000873
3,2013-01-01 03:00:00,0.000928,0.002503,0.002607,0.137212,0.002074,0.009557,0.001121,0.831023,0.938460,...,0.001190,0.864818,0.927961,0.386180,0.001006,0.000000,0.036644,0.001833,0.262663,0.000873
4,2013-01-01 04:00:00,0.000000,0.000000,0.002607,0.130745,0.002074,0.010375,0.001121,0.645404,0.759356,...,0.001190,0.864818,0.929391,0.385410,0.001006,0.000000,0.041910,0.001833,0.264453,0.000873
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2013-12-31 19:00:00,0.029153,0.052263,0.000961,0.111495,0.002074,0.001800,0.000976,0.919508,0.989035,...,0.000900,0.774638,0.858609,0.307886,0.001006,0.025999,0.003614,0.001377,0.074752,0.000940
8756,2013-12-31 20:00:00,0.029161,0.065263,0.000961,0.129865,0.002074,0.001800,0.000976,0.778407,0.910707,...,0.002864,0.737556,0.855220,0.283948,0.001006,0.021950,0.005001,0.001377,0.082766,0.000940
8757,2013-12-31 21:00:00,0.029153,0.070495,0.001175,0.129865,0.002074,0.001812,0.000976,0.707040,0.847579,...,0.010672,0.692466,0.791745,0.247635,0.001006,0.000050,0.005051,0.001377,0.083883,0.000940
8758,2013-12-31 22:00:00,0.029545,0.052263,0.001175,0.161829,0.002074,0.002011,0.000976,0.766593,0.919846,...,0.000017,0.672129,0.785177,0.204160,0.001006,0.000000,0.008033,0.001377,0.090750,0.000940


In [20]:
capfacts_pred_q40.dtypes

snapshot             object
AL0 0 offwind-ac    float64
AL0 0 onwind        float64
AL0 0 solar         float64
AT0 0 onwind        float64
                     ...   
SI0 0 offwind-ac    float64
SI0 0 onwind        float64
SI0 0 solar         float64
SK0 0 onwind        float64
SK0 0 solar         float64
Length: 126, dtype: object